In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
from PIL import Image
from PIL import ImageFile
import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.platform import app
from delf import delf_config_pb2
from delf import feature_io
from delf.python.detect_to_retrieve import dataset
from delf import extract_features

import traceback


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
# Extensions.
_DELF_EXTENSION = '.delf'

OUTPUT_PATH = 'output/test'
IMAGE_DIR = 'image_test'

# To avoid PIL crashing for truncated (corrupted) images.
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
def _PilLoader(path):
    """Helper function to read image with PIL.

    Args:
        path: Path to image to be loaded.

    Returns:
        PIL image in RGB format.
    """
    with tf.gfile.GFile(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


In [4]:
# Read list of query images from dataset file.
tf.logging.info('Reading list of query images and boxes from dataset file...')
query_list = os.listdir(IMAGE_DIR)
num_images = len(query_list)
tf.logging.info('done! Found %d images', num_images)


INFO:tensorflow:Reading list of query images and boxes from dataset file...
INFO:tensorflow:done! Found 5879 images


In [5]:
# Parse DelfConfig proto.
config = delf_config_pb2.DelfConfig()
with tf.gfile.GFile('delf_gld_config.pbtxt', 'r') as f:
    text_format.Merge(f.read(), config)

# Create output directory if necessary.
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [6]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        # Initialize variables, construct DELF extractor.
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        extractor_fn = extract_features.MakeExtractor(sess, config)

        start = time.clock()
        for i in range(num_images):
            
            try:
            
                query_image_name = query_list[i]
                input_image_filename = os.path.join(IMAGE_DIR,query_image_name)
                output_feature_filename = os.path.join(OUTPUT_PATH, query_image_name + _DELF_EXTENSION)
                if tf.gfile.Exists(output_feature_filename):
                    tf.logging.info('Skipping %s', query_image_name)
                    continue

                # Crop query image according to bounding box.
                # bbox = [int(round(b)) for b in ground_truth[i]['bbx']]
                # im = np.array(_PilLoader(input_image_filename).crop(bbox))

                im = np.array(_PilLoader(input_image_filename))

                # Extract and save features.
                (locations_out, descriptors_out, feature_scales_out,attention_out) = extractor_fn(im)

                feature_io.WriteToFile(output_feature_filename, locations_out,feature_scales_out, descriptors_out,attention_out)

                elapsed = (time.clock() - start)
                
                if i % 10 == 0 :
                    print('Processed %d query images in %f seconds' % (i, elapsed))
            
            except :
                traceback.print_exc()


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from parameters/delf_gld_20190411/model\variables\variables
Instructions for updating:
dim is deprecated, use axis instead
INFO:tensorflow:Skipping 00016575233bc956.jpg
INFO:tensorflow:Skipping 0001aadbcd8cb923.jpg
INFO:tensorflow:Skipping 0002c06b2440a5f9.jpg
INFO:tensorflow:Skipping 0002eb1ee5a5a6b2.jpg
INFO:tensorflow:Skipping 000594dad986513e.jpg
INFO:tensorflow:Skipping 0006b7eb42bc7ee2.jpg
INFO:tensorflow:Skipping 00084cdf8f600d00.jpg
INFO:tensorflow:Skipping 0008722c48c52fda.jpg


C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Skipping 0009ffd36a94a88f.jpg
INFO:tensorflow:Skipping 000a1ee0f98496d7.jpg
INFO:tensorflow:Skipping 000b15b043eb8cf0.jpg
INFO:tensorflow:Skipping 000b90e5adff9e69.jpg
INFO:tensorflow:Skipping 000c7a9f133a31ba.jpg


C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Processed 20 query images in 16.984258 seconds
Processed 30 query images in 21.596531 seconds
Processed 40 query images in 28.033467 seconds
Processed 50 query images in 34.012216 seconds
Processed 60 query images in 40.532921 seconds
Processed 70 query images in 47.518607 seconds
Processed 80 query images in 52.510263 seconds
Processed 90 query images in 56.154303 seconds
Processed 100 query images in 61.137005 seconds
Processed 110 query images in 64.978770 seconds
Processed 120 query images in 70.785472 seconds
Processed 130 query images in 75.724053 seconds
Processed 140 query images in 78.323418 seconds
Processed 150 query images in 83.121573 seconds
Processed 160 query images in 85.769423 seconds
Processed 170 query images in 94.218694 seconds
Processed 180 query images in 99.210057 seconds
Processed 190 query images in 104.291280 seconds
Processed 200 query images in 108.971527 seconds
Processed 210 query images in 114.908080 seconds
Processed 220 query images in 117.697855 seco

Processed 1690 query images in 502.135267 seconds
Processed 1700 query images in 503.223163 seconds
Processed 1710 query images in 504.788293 seconds
Processed 1720 query images in 506.272623 seconds
Processed 1730 query images in 509.307723 seconds
Processed 1740 query images in 512.652077 seconds
Processed 1750 query images in 513.704692 seconds
Processed 1760 query images in 515.961425 seconds
Processed 1770 query images in 519.312890 seconds
Processed 1780 query images in 521.679020 seconds
Processed 1790 query images in 523.323094 seconds
Processed 1800 query images in 525.333447 seconds
Processed 1810 query images in 526.835100 seconds
Processed 1820 query images in 529.621982 seconds
Processed 1830 query images in 532.285979 seconds
Processed 1840 query images in 535.106949 seconds
Processed 1850 query images in 537.214379 seconds
Processed 1860 query images in 538.963995 seconds
Processed 1870 query images in 540.296480 seconds
Processed 1880 query images in 541.313329 seconds


Processed 3330 query images in 791.934423 seconds
Processed 3340 query images in 794.039013 seconds
Processed 3350 query images in 795.403747 seconds
Processed 3360 query images in 796.625781 seconds
Processed 3370 query images in 798.891347 seconds
Processed 3380 query images in 800.970248 seconds
Processed 3390 query images in 804.059277 seconds
Processed 3400 query images in 805.319443 seconds
Processed 3410 query images in 806.497118 seconds
Processed 3420 query images in 808.840102 seconds
Processed 3430 query images in 810.999421 seconds
Processed 3440 query images in 811.980108 seconds
Processed 3450 query images in 813.006149 seconds
Processed 3460 query images in 814.230992 seconds
Processed 3470 query images in 815.556405 seconds
Processed 3480 query images in 817.035740 seconds
Processed 3490 query images in 818.603059 seconds
Processed 3500 query images in 820.019748 seconds
Processed 3510 query images in 821.556481 seconds
Processed 3520 query images in 824.015177 seconds


Processed 4970 query images in 1065.827218 seconds
Processed 4980 query images in 1067.830318 seconds
Processed 4990 query images in 1070.752536 seconds
Processed 5000 query images in 1072.715192 seconds
Processed 5010 query images in 1074.359140 seconds
Processed 5020 query images in 1075.602369 seconds
Processed 5030 query images in 1077.229888 seconds
Processed 5040 query images in 1078.495864 seconds
Processed 5050 query images in 1081.208709 seconds
Processed 5060 query images in 1083.821804 seconds
Processed 5070 query images in 1084.813216 seconds
Processed 5080 query images in 1086.376570 seconds
Processed 5090 query images in 1087.620674 seconds
Processed 5100 query images in 1088.761447 seconds
Processed 5110 query images in 1090.065628 seconds
Processed 5120 query images in 1091.481481 seconds
Processed 5130 query images in 1093.013566 seconds
Processed 5140 query images in 1094.514332 seconds
Processed 5150 query images in 1096.234490 seconds
Processed 5160 query images in 